In [1]:
import json
import psycopg2
import pandas as pd
from psycopg2.extras import execute_batch
from pathlib import Path
from sentence_transformers import SentenceTransformer
from psycopg2.extras import execute_values

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Database connection parameters
DB_PARAMS = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",
    "port": "5433"  # Default is 5432
}

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(**DB_PARAMS)
    conn.autocommit = True
    # cursor = conn.cursor()
except Exception as e:
    print(f"Error connecting to database: {e}")
    exit(1)

init_pgvector = "CREATE EXTENSION IF NOT EXISTS vector;" # execute this first
create_table = """CREATE TABLE "example1" (id bigserial PRIMARY KEY, keyword VARCHAR(100), embedding vector(384));"""
insert_embeddings = "INSERT INTO example1 (keyword, embedding) VALUES %s"

c:\Users\User\alejo\vector_db\vector_db\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
#install pgvector
cur = conn.cursor()
cur.execute(init_pgvector)
cur.execute(create_table)
conn.commit()
cur.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cur.fetchall())


[('example',), ('example1',)]


In [2]:
#limite maximo es de 384 tokens
sentences = ["hello world", "hello python", "bye python", "la girafa es roja", "python", "bython", "el mico esta trepando", "mico", "el mico es cafe"]
embeddings = model.encode(sentences, normalize_embeddings=True).tolist()

In [3]:
print(len(sentences))
print(len(embeddings[0]))

9
384


In [4]:
# embeddings = model.encode(sentences, normalize_embeddings=True).tolist()
data = [(sentence, embedding) for sentence, embedding in zip(sentences, embeddings)]
with conn.cursor() as curs:
    execute_values(
                curs,
                insert_embeddings,
                data,
                template="(%s, %s)"
            )

In [5]:
data = str(model.encode("python", normalize_embeddings=False).tolist())
with conn.cursor() as curs:
    curs.execute("""
            SELECT keyword,
            1-(embedding <=> %(data)s) as similarity
            FROM example
            --WHERE 1-(embedding <=> %(data)s) >= 0.5
            ORDER BY embedding <=> %(data)s
            LIMIT 5;
        """,
        {"data": data}
    )
    result = [(keyword) for keyword in curs.fetchall()]
result

[('python', 1.0),
 ('python', 1.0),
 ('hello python', 0.8450508713722229),
 ('hello python', 0.8450508713722229),
 ('bye python', 0.6831845045089722)]

In [6]:
with conn.cursor() as curs:
    curs.execute("CREATE INDEX ON example USING ivfflat (embedding vector_cosine_ops) WITH (lists = 100);")

In [25]:
cur = conn.cursor()
cur.execute("SELECT * FROM example1")
tuples_list = cur.fetchall()
#print(tuples_list)

df = pd.DataFrame(tuples_list, columns=["id","keyword","vector"])
df.head(5)

,id,keyword,vector
0,1,hello world,"[-0.03447727,0.031023229,0.006734971,0.0261089..."
1,2,hello python,"[-0.050150678,0.027626721,-0.014199368,0.05738..."
2,3,bye python,"[-0.0173582,0.055295315,0.044234786,0.05157163..."
3,4,la girafa es roja,"[-0.017561583,0.027168112,-0.05538615,0.040346..."
4,5,python,"[-0.05615145,0.017742552,-0.059133362,0.040266..."


In [52]:
from ast import literal_eval

df["vector"] = df.vector.apply(literal_eval)

df["vector_length"] = df["vector"].apply(len)

ValueError: malformed node or string: [-0.03447727, 0.031023229, 0.006734971, 0.026108956, -0.039362054, -0.1603025, 0.06692399, -0.0064414283, -0.04745049, 0.014758861, 0.070875295, 0.05552758, 0.019193375, -0.026251333, -0.01010954, -0.026940506, 0.022307457, -0.022226686, -0.1496926, -0.017493034, 0.0076762526, 0.054352295, 0.0032544436, 0.031725906, -0.08462135, -0.029406007, 0.05159558, 0.048124082, -0.0033147954, -0.058279198, 0.04196929, 0.022210686, 0.12818883, -0.022338957, -0.011656217, 0.062928334, -0.03287631, -0.091226056, -0.031175377, 0.05269955, 0.0470348, -0.084203094, -0.030056167, -0.02074481, 0.009517839, -0.0037218125, 0.007343314, 0.039324336, 0.093274035, -0.0037885893, -0.052742094, -0.05805821, -0.0068643717, 0.0052832714, 0.08289297, 0.019362831, 0.006284503, -0.010330768, 0.009032373, -0.037683733, -0.04520607, 0.024016323, -0.006944155, 0.013491643, 0.10005486, -0.07168391, -0.021695092, 0.03161841, -0.051634617, -0.082247764, -0.06569329, -0.009895369, 0.00

In [ ]:
df.head(5)

,id,keyword,vector,vector_length
0,1,hello world,"[-0.03447727, 0.031023229, 0.006734971, 0.0261...",384
1,2,hello python,"[-0.050150678, 0.027626721, -0.014199368, 0.05...",384
2,3,bye python,"[-0.0173582, 0.055295315, 0.044234786, 0.05157...",384
3,4,la girafa es roja,"[-0.017561583, 0.027168112, -0.05538615, 0.040...",384
4,5,python,"[-0.05615145, 0.017742552, -0.059133362, 0.040...",384
